In [ ]:
# TODO: Remove fixed numpy version once rdkit supports numpy>=2.0.0
%pip install biopython numpy==1.26.4 pandas rdkit tqdm

In [ ]:
# General configuration

import os
import sys

import pandas as pd
from tqdm import tqdm

sys.path.append(os.path.abspath("src"))
from dfcompare import DFCompare
from helper import mkdir, path, copyfile
from pdbbind import PDBbind

DATASET_YEAR = 2007
PDBBIND_USER = os.environ.get("PDBBIND_USER")
PDBBIND_PASS = os.environ.get("PDBBIND_PASS")

pd.options.display.width = 120
pd.options.display.max_rows = 50
pd.options.display.float_format = "{:.2f}".format

In [ ]:
# Path management

INPUT_PATH = mkdir(f"input/{DATASET_YEAR}")
ORIG_PATH = "../PLBAffinity/CASF/"

AA_PATH = path(f"{INPUT_PATH}/CASF_{DATASET_YEAR}_aa.csv")
LIGAND_PATH = path(f"{INPUT_PATH}/CASF_{DATASET_YEAR}_ligprop.csv")
ACT_PATH = path(f"{INPUT_PATH}/CASF_{DATASET_YEAR}_activities.csv")

ORIG_AA_PATH = path(f"{ORIG_PATH}/{AA_PATH}")
ORIG_LIGAND_PATH = path(f"{ORIG_PATH}/{LIGAND_PATH}")
ORIG_ACT_TEST_PATH = path(f"{ORIG_PATH}/{ACT_PATH}").replace(".csv", "_test.csv")
ORIG_ACT_TRAIN_PATH = path(f"{ORIG_PATH}/{ACT_PATH}").replace(".csv", "_train.csv")

# NOTE: The original input for the year 2019 has different filenames and no test/train split
if DATASET_YEAR in [2019, 2020]:
    ORIG_INPUT_PATH = path(f"{ORIG_PATH}/{INPUT_PATH}")
    ORIG_AA_PATH = path(f"{ORIG_INPUT_PATH}/refined_{DATASET_YEAR}_aa.csv")
    ORIG_ACT_TEST_PATH = path(f"{ORIG_INPUT_PATH}/PDBind_{DATASET_YEAR}_activities.csv")
    ORIG_ACT_TRAIN_PATH = ORIG_ACT_TEST_PATH

# NOTE: We stub the nonexistent original 2020 dataset with the 2019 dataset
if DATASET_YEAR == 2020:
    mkdir(ORIG_INPUT_PATH)
    copyfile(ORIG_LIGAND_PATH.replace("2020", "2019"), ORIG_LIGAND_PATH)
    copyfile(ORIG_AA_PATH.replace("2020", "2019"), ORIG_AA_PATH)
    copyfile(ORIG_ACT_TEST_PATH.replace("2020", "2019"), ORIG_ACT_TEST_PATH)

In [ ]:
# Data preparation

CASF_IS_AVAILABLE = DATASET_YEAR in [2007, 2013, 2016]
pdbbind = PDBbind("datasets", PDBBIND_USER, PDBBIND_PASS)
pdbbind.prepare(DATASET_YEAR, "casf") if CASF_IS_AVAILABLE else None
pdbbind.prepare(DATASET_YEAR, "refined")

In [ ]:
# NOTE: All values taken from btad502_supplementary_data.pdf

# fmt: off

AMINO_ACIDS = [
    "ALA", "ARG", "ASN", "ASP", "CYS", "GLN", "GLU", "GLY", "ILE", "LEU",
    "LYS", "MET", "PHE", "PRO", "SER", "THR", "TRP", "TYR", "VAL", "HIS"]

LIGAND_DESCRIPTORS = [
    "Descriptors.TPSA",
    "Descriptors.MolLogP",
    "Descriptors.MolWt",
    "Lipinski.HeavyAtomCount",
    "Lipinski.NHOHCount",
    "Lipinski.NOCount",
    "Lipinski.NumHAcceptors",
    "Lipinski.NumHDonors",
    "Lipinski.NumHeteroatoms",
    "Lipinski.NumRotatableBonds",
    "Descriptors.NumValenceElectrons",
    "rdMolDescriptors.CalcNumAmideBonds",
    "Lipinski.NumAromaticRings",
    "Lipinski.NumSaturatedRings",
    "Lipinski.NumAliphaticRings",
    "Lipinski.NumAromaticHeterocycles",
    "Lipinski.NumAromaticCarbocycles",
    "Lipinski.NumSaturatedHeterocycles"]

INCOMP_LIGANDS = {
  2007: ["1a7x","1b8n","1ksn","1mue","1nw5","1rle","1sl3","1tyr","2adm","2j7h"],
  2013: ["1nw5","1wc1","2h21","2j7h","3acl","3dx2","3gcp","3l4u","3l4v","3l4x",
         "3l4y","3l4z","3lcv","3lpp","3pgl","3r24"],
  2016: ["1nw5","1wc1","2h21","2j7h","3acl","3gcp","3l4u","3l4v","3l4x","3l4y",
         "3l4z","3lcv","3lpp","3pgl","3r24","4bup","4qhc","4qy3","4ufi","4ymg",
         "4zbf"],
  2019: ["4qy3","3t85","4zbf","3ta0","6d55","3ta1","6eqx","3t70","6chp","3tao",
         "5fyx","3tay","6cze","6bm5","2h21","6dh6","3l4v","6dj7","3tb6","5h5f",
         "6dh8","6ced","6csq","6dq4","6cwn","6css","3acl","3t84","3t82","6cjv",
         "3l4u","5twj","6d5e","6d2o","4ufi","6d5j","6czb","6cfc","3tcg","3td4",
         "4bup","3pgl","3lpp","3t8v","6dj2","6d9x","3l4x","3gcp","2j7h","5bw4",
         "3r24","4ymg","6cn5","1nw5","3t83","1wc1","6dil","6dh1","6d56","6e4a",
         "6dif","6dai","3l4y","3l4z","5kva","6csr","6cwh","6dh7"],
  2020: [""]
}

# fmt: on

In [ ]:
import warnings
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.PDBExceptions import PDBConstructionWarning

warnings.simplefilter("ignore", PDBConstructionWarning)

pockets = pdbbind.pockets(DATASET_YEAR, "refined")

pocket_values = []
for pocket, filename in tqdm(pockets.items()):  # type: ignore
    aa_counts = {aa: 0 for aa in AMINO_ACIDS}

    pdb_parser = PDBParser()
    structure = pdb_parser.get_structure(pocket, filename)

    for model in structure:
        for chain in model:
            for residue in chain:
                res_name = residue.get_resname()
                if res_name in AMINO_ACIDS:
                    aa_counts[res_name] += 1

    pocket_values.append([pocket] + list(aa_counts.values()))

df = pd.DataFrame(pocket_values)
df.sort_values(by=df.columns[0], inplace=True)

df.to_csv(AA_PATH, index=False, sep=",")

In [ ]:
print(ORIG_AA_PATH)
df1 = pd.read_csv(ORIG_AA_PATH, header=None, index_col=0, skiprows=1)
df2 = pd.read_csv(AA_PATH, header=None, index_col=0, skiprows=1)

dfcompare = DFCompare(df1, df2)
dfcompare.print_report("pocket")

In [ ]:
from rdkit import Chem
from rdkit import RDLogger
import rdkit.Chem.Descriptors as Descriptors
import rdkit.Chem.rdMolDescriptors as rdMolDescriptors
import rdkit.Chem.Lipinski as Lipinski


logger = RDLogger.logger()
logger.setLevel(RDLogger.CRITICAL)  # RDLogger.ERROR


def calculate_descriptors(mol, descriptor_list):
    results = {}
    for desc in descriptor_list:
        module_name, func_name = desc.split(".")
        module = globals()[module_name]
        desc_func = getattr(module, func_name)
        results[desc] = desc_func(mol)
    return results


def print_ligand_values(values):
    fmt_values = [f"{v:.2f}" if isinstance(v, float) else str(v) for v in values]
    print(",".join(fmt_values))


ligands = pdbbind.ligands(DATASET_YEAR, "casf") if CASF_IS_AVAILABLE else {}
ligands = ligands | pdbbind.ligands(DATASET_YEAR, "refined")  # type: ignore

invalid_ligands = []
ligands_values = []
for ligand, filename in tqdm(ligands.items()):
    # supplier = Chem.SDMolSupplier(filename)
    # mol = supplier[0] if supplier else None
    mol = Chem.MolFromMol2File(filename)  # type: ignore
    if not mol:
        invalid_ligands.append(ligand)
        continue

    descriptor_values = calculate_descriptors(mol, LIGAND_DESCRIPTORS)
    ligands_values.append([ligand] + list(descriptor_values.values()))
    # print_ligand_values(ligands_values[-1])

df = pd.DataFrame(ligands_values)
df.sort_values(by=df.columns[0], inplace=True)
float_cols = df.select_dtypes(include=["float"]).columns
df[float_cols] = df[float_cols].map("{:.2f}".format)

df.to_csv(LIGAND_PATH, index=False, sep=",")

print("Invalid ligands:", f"[{len(invalid_ligands)}]", ",".join(invalid_ligands))

In [ ]:
# NOTE: NumRotatableBonds seems to be incorrect in original, so we use ignore_rows=[10]
# https://pubchem.ncbi.nlm.nih.gov/rest/pug/substance/name/1MRS/cids/XML?list_return=grouped
# https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/447206/property/RotatableBondCount/XML

print(
    f"Incompatible ligands in {DATASET_YEAR}:",
    f"[{len(INCOMP_LIGANDS[DATASET_YEAR])}]",
    ",".join(INCOMP_LIGANDS[DATASET_YEAR]),
)

df1 = pd.read_csv(ORIG_LIGAND_PATH, header=None, index_col=0, skiprows=1)
df2 = pd.read_csv(LIGAND_PATH, header=None, index_col=0, skiprows=1)

dfcompare = DFCompare(df1, df2)
dfcompare.print_report("ligand", ignore_rows=[10])

In [ ]:
activity_values = pdbbind.activities(DATASET_YEAR, "refined")

df = pd.DataFrame(activity_values)
df.sort_values(by=df.columns[0], inplace=True)

# TODO: Fix the values rather than removing them
df = df[~df.iloc[:, 0].isin(invalid_ligands)]

df.to_csv(ACT_PATH, index=False, sep=",")

In [ ]:
# NOTE: We do the train/test split later on
print(ORIG_ACT_TEST_PATH)
df1_test = pd.read_csv(ORIG_ACT_TEST_PATH, header=None, index_col=0, skiprows=0)
df1_train = pd.read_csv(ORIG_ACT_TRAIN_PATH, header=None, index_col=0, skiprows=0)

test_train_split = ORIG_ACT_TEST_PATH != ORIG_ACT_TRAIN_PATH
df1 = pd.concat([df1_test, df1_train]) if test_train_split else df1_test
df2 = pd.read_csv(ACT_PATH, header=None, index_col=0, skiprows=1)

dfcompare = DFCompare(df1, df2)
dfcompare.print_report("activity")